In [1]:
from implicit.evaluation import  *
from implicit.als import AlternatingLeastSquares as ALS
from implicit.bpr import BayesianPersonalizedRanking as BPR
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from sklearn.utils import shuffle
from scipy.sparse import *
# import rec_util
pd.options.display.float_format = '{:.2f}'.format
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
song_meta = pd.read_json('./raw_data/song_meta.json')
train = pd.read_json('./raw_data/train.json')
val = pd.read_json('./raw_data/val.json')
genre = pd.read_json('./raw_data/genre_gn_all.json', typ = 'seriese')

In [3]:
song_meta.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4


## id별 장르 구분

In [4]:
song_meta[['id','song_gn_dtl_gnr_basket','song_gn_gnr_basket']]

,id,song_gn_dtl_gnr_basket,song_gn_gnr_basket
0,0,[GN0901],[GN0900]
1,1,"[GN1601, GN1606]",[GN1600]
2,2,[GN0901],[GN0900]
3,3,"[GN1102, GN1101]",[GN1100]
4,4,"[GN1802, GN1801]",[GN1800]
...,...,...,...
707984,707984,[GN2001],[GN2000]
707985,707985,[GN0901],[GN0900]
707986,707986,"[GN0105, GN0101]",[GN0100]
707987,707987,"[GN1807, GN1801]",[GN1800]


In [5]:
song_meta['total_gn'] = song_meta['song_gn_dtl_gnr_basket'] + song_meta['song_gn_gnr_basket']

In [6]:
song_meta[['id','total_gn']]

,id,total_gn
0,0,"[GN0901, GN0900]"
1,1,"[GN1601, GN1606, GN1600]"
2,2,"[GN0901, GN0900]"
3,3,"[GN1102, GN1101, GN1100]"
4,4,"[GN1802, GN1801, GN1800]"
...,...,...
707984,707984,"[GN2001, GN2000]"
707985,707985,"[GN0901, GN0900]"
707986,707986,"[GN0105, GN0101, GN0100]"
707987,707987,"[GN1807, GN1801, GN1800]"


In [7]:
df = song_meta[['id','total_gn']].explode('total_gn')
# df1.explode('song_gn_dtl_gnr_basket')
df

,id,total_gn
0,0,GN0901
0,0,GN0900
1,1,GN1601
1,1,GN1606
1,1,GN1600
...,...,...
707987,707987,GN1801
707987,707987,GN1800
707988,707988,GN0601
707988,707988,GN0604


# 이진행렬 만들기

In [8]:
df1 = pd.get_dummies(df['total_gn']).max(level=0)

In [9]:
df1.shape

(707989, 249)

## 빠진 장르 확인

In [10]:
a = df1.columns.difference(['id']).tolist()

In [11]:
b = genre.index.tolist()

In [12]:
s = set(a)
temp3 = [x for x in b if x not in s]

In [13]:
temp3

['GN0305', 'GN1612', 'GN1709', 'GN2110', 'GN2604', 'GN3000']

In [14]:
genre_df = pd.DataFrame(genre).reset_index()
genre_df

,index,0
0,GN0100,발라드
1,GN0101,세부장르전체
2,GN0102,'80
3,GN0103,'90
4,GN0104,'00
...,...,...
249,GN2900,뮤지컬
250,GN2901,세부장르전체
251,GN2902,국내뮤지컬
252,GN2903,국외뮤지컬


In [15]:
genre_df[genre_df['index'] == 'GN3000']

,index,0
253,GN3000,크리스마스


In [16]:
tr_csr = csr_matrix(df1.values)

In [17]:
tr_csr

<707989x249 sparse matrix of type '<class 'numpy.uint8'>'
	with 2315254 stored elements in Compressed Sparse Row format>

In [18]:
# # 메모리 터짐
# from sklearn.metrics import pairwise_distances
# pairwise_matrix = pairwise_distances(tr_csr, tr_csr, n_jobs=-1)

In [19]:
# model 1

In [20]:
als_model = ALS(factors=256, regularization=0.08)

## 모델링
- 노래 100개 먼저 채우기

In [21]:
als_model.fit(tr_csr * 32)

In [22]:
from numba import jit

@jit
def similar_songs(song_list):
    ls = []
    for song in song_list:
        ls += als_model.similar_items(song, N=100)[:int(100 / len(song_list)) + 1]
    return [i[0] for i in ls][:100]

# 노래 먼저 100개 채우기, 약 5시간 걸림

In [23]:
val_song_100 = val.copy()

In [24]:
val_song_100['songs_predicted'] = val_song_100['songs'].progress_apply(similar_songs)

100%|██████████| 23015/23015 [4:58:19<00:00,  1.29it/s]  


In [25]:
val_song_100.to_json('./val_song_100.json')

## tag에 장르 붙이기

In [24]:
tagandsong = train.loc[:,['tags','songs']]

In [72]:
df = song_meta[['id','total_gn']].explode('total_gn')
# df1.explode('song_gn_dtl_gnr_basket')

,id,total_gn
0,0,GN0901
0,0,GN0900
1,1,GN1601
1,1,GN1606
1,1,GN1600
...,...,...
707987,707987,GN1801
707987,707987,GN1800
707988,707988,GN0601
707988,707988,GN0604


In [49]:
ls = []
for idx, x in tagandsong.iloc[:1].iterrows():
    for tag in x['tags']:
        for song in x['songs']:
            ls.append({'tag' : tag,
            'genre': df[df['id'] == song]['total_gn'].tolist()})

In [73]:
def song_tag_match(x):
    ls = []
    for tag in x['tags']:
        for song in x['songs']:
            dt = {'tag' : tag,
            'genre': df[df['id'] == song]['total_gn'].tolist()}
            ls.append(dt)
            dt = {}
    return ls

# 12시간 걸림

In [75]:
tagandsong.shape

(115071, 2)

In [74]:
result = tagandsong.iloc[:10].apply(lambda x: song_tag_match(x), axis=1)

In [68]:
result_df = pd.DataFrame(result.sum())

In [69]:
result_df.groupby('tag').agg(sum)

,genre
tag,
2017,"[GN0901, GN0900, GN1201, GN1200, GN1105, GN110..."
Metal,"[GN0601, GN0605, GN0600, GN0601, GN0605, GN060..."
M에센셜,"[GN0601, GN0605, GN0600, GN0601, GN0605, GN060..."
Pop,"[GN0901, GN0900, GN1201, GN1200, GN1105, GN110..."
Rock,"[GN0601, GN0605, GN0600, GN0601, GN0605, GN060..."
겨울노래,"[GN0908, GN1509, GN0901, GN2207, GN1501, GN150..."
겨울왕국,"[GN0908, GN1509, GN0901, GN2207, GN1501, GN150..."
고백,"[GN0101, GN0100, GN0401, GN0403, GN0402, GN040..."
기분전환,"[GN0901, GN0900, GN1201, GN1200, GN1105, GN110..."


In [85]:
val_song_100[(val_song_100['tags'].apply(len) == 0) & (val_song_100['songs'].apply(len) == 0)]

,tags,id,plylst_title,songs,like_cnt,updt_date
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000
35,[],65114,"■■■■ 사랑,그리고이별 ■■■■",[],6,2010-10-27 10:34:34.000
57,[],87700,마쉬멜로우같은 멜로우한 음악,[],6,2016-01-14 10:19:30.000
71,[],35271,공부와 독서를 위한 #Newage,[],10,2020-01-17 15:46:20.000
...,...,...,...,...,...,...
22903,[],140513,10년이 지나 들어도 좋은 감성 Ballad,[],405,2016-01-11 10:58:05.000
22920,[],124704,가사의 의미와 뜻은모른다!! 오직 멜로디로만 선곡한 팝송!!,[],27,2016-02-05 12:31:59.000
22981,[],13045,＊카페 느낌 샹송♭,[],38,2011-07-12 00:58:39.000
22991,[],32537,컨트리 황제 조니 캐시가 선 레코드 시절 발표한 초기 대표작,[],28,2019-06-17 14:22:48.000
